# Example Sample Hotel and Flight Booker Agent 

এই সমাধানটি আপনাকে ফ্লাইট টিকেট এবং হোটেল বুক করতে সাহায্য করবে।  দৃশ্যটি হলো একটি যাত্রা লন্ডন হিথ্রো LHR থেকে ফেব্রুয়ারি ২০, ২০২৪ তারিখে নিউ ইয়র্ক JFK গিয়ে ফেব্রুয়ারি ২৭, ২০২৫ তারিখে ফিরে আসা, ইকোনমি ক্লাসে শুধুমাত্র ব্রিটিশ এয়ারওয়েজ দ্বারা ভ্রমণ। আমি নিউ ইয়র্কে একটি হিল্টন হোটেলে থাকার ইচ্ছা রাখি, দয়া করে.flight এবং হোটেলের খরচ প্রদান করুন।


# Azure AI এজেন্ট সার্ভিস ইনিশিয়ালাইজ করুন এবং **.env** থেকে কনফিগারেশন তথ্য পান

### **.env**

একটি .env ফাইল তৈরি করুন

**.env**-এ Azure AI এজেন্ট সার্ভিসের সংযোগ স্ট্রিং, AOAI দ্বারা ব্যবহৃত মডেল, এবং সংশ্লিষ্ট Google API সার্চ সার্ভিসের API, ENDPOINT ইত্যাদি থাকে।

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "আপনার Azure AI এজেন্ট সার্ভিস মডেল ডিপ্লয়মেন্টের নাম"

[**NOTE**] আপনার এমন একটি মডেল প্রয়োজন যার 100,000 রেট লিমিট (প্রতি মিনিট টোকেন) এবং 600 রেট লিমিট (প্রতি মিনিট অনুরোধ)

  আপনি মডেলটি Microsoft Foundry - Model and Endpoint থেকে পেতে পারেন।

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "আপনার Azure AI এজেন্ট সার্ভিস প্রজেক্ট সংযোগ স্ট্রিং"

  আপনি AI ​​Foundry পোর্টাল স্ক্রিনে আপনার প্রজেক্ট ওভারভিউ থেকে প্রজেক্ট সংযোগ স্ট্রিং পেতে পারেন।

- **SERPAPI_SEARCH_API_KEY** = "আপনার SERPAPI সার্চ API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "আপনার SERPAPI সার্চ Endpoint"

Azure AI এজেন্ট সার্ভিসের মডেল ডিপ্লয়মেন্ট নাম এবং প্রজেক্ট সংযোগ স্ট্রিং পেতে, আপনাকে Azure AI এজেন্ট সার্ভিস তৈরি করতে হবে। সরাসরি তৈরি করতে [এই টেমপ্লেটটি](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) ব্যবহার করার পরামর্শ দেওয়া হয়েছে (***দ্রষ্টব্য:*** Azure AI এজেন্ট সার্ভিস বর্তমানে সীমিত অঞ্চলে সেট করা আছে। অঞ্চল নির্ধারণের জন্য [এই লিঙ্কটি](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) দেখার পরামর্শ দেওয়া হয়)

এজেন্টকে SERPAPI অ্যাক্সেস করতে হবে। [এই লিঙ্ক](https://serpapi.com/searches) ব্যবহার করে নিবন্ধন করার পরামর্শ দেওয়া হয়। নিবন্ধনের পরে, আপনি একটি অনন্য API KEY এবং ENDPOINT পেতে পারবেন।


# সেটআপ 

এই নোটবুক চালাতে, আপনাকে নিশ্চিত করতে হবে যে আপনি প্রয়োজনীয় লাইব্রেরিগুলি ইনস্টল করেছেন `pip install -r requirements.txt` চালিয়ে।


In [ ]:
from semantic_kernel import __version__

__version__

আপনার Semantic Kernel সংস্করণ কমপক্ষে ১.২৭.২ হতে হবে।


আপনার .env ফাইল সেটিং এবং রিসোর্স লোড করুন দয়া করে নিশ্চিত করুন যে আপনি আপনার কী এবং সেটিং যোগ করেছেন এবং একটি লোকাল .env ফাইল তৈরি করেছেন।


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure এ লগ ইন করুন

আপনাকে এখন Azure এ লগ ইন করতে হবে। একটি টার্মিনাল খুলুন এবং নিম্নলিখিত কমান্ডটি চালান:

```bash
az login
```

এই কমান্ডটি আপনাকে আপনার Azure শংসাপত্র প্রবেশ করার জন্য অনুরোধ করবে, যা Azure AI Agent পরিষেবাটিকে সঠিকভাবে কাজ করতে সক্ষম করে।


# ব্যাখ্যা:
এটি একটি ভেরিয়েবল যা SERP (Search Engine Results Page) API পরিষেবায় অ্যাক্সেস করার জন্য API কী সংরক্ষণ করে। একটি API কী একটি অনন্য শনাক্তকারী যা আপনার অ্যাকাউন্টের সাথে যুক্ত অনুরোধগুলি প্রমাণীকরণের জন্য ব্যবহৃত হয়।

উদ্দেশ্য: এই লাইনের উদ্দেশ্য হল API কী একটি ভেরিয়েবলে সংরক্ষণ করা যাতে এটিকে SERP API পরিষেবায় অনুরোধগুলি প্রমাণীকরণের জন্য ব্যবহার করা যায়। পরিষেবায় প্রবেশ করতে এবং অনুসন্ধান সম্পাদন করতে API কী প্রয়োজন।
কিভাবে একটি SERP API কী পাবেন: একটি SERP API কী পেতে, সাধারণভাবে https://serpapi.com-এ এই ধাপগুলি অনুসরণ করুন (নির্দিষ্ট SERP API পরিষেবার উপর নির্ভর করে সঠিক ধাপগুলি ভিন্ন হতে পারে):

SERP API পরিষেবা নির্বাচন করুন: বেশ কয়েকটি SERP API পরিষেবা উপলব্ধ, যেমন SerpAPI, Google Custom Search JSON API, এবং অন্যান্য। আপনার প্রয়োজন অনুসারে সবচেয়ে উপযুক্তটি নির্বাচন করুন।

একটি অ্যাকাউন্টের জন্য সাইন আপ করুন: নির্বাচিত SERP API পরিষেবার ওয়েবসাইটে যান এবং একটি অ্যাকাউন্টের জন্য সাইন আপ করুন। কিছু মৌলিক তথ্য প্রদান এবং আপনার ইমেইল ঠিকানা যাচাই করতে হতে পারে।

API কী তৈরি করুন: সাইন আপ করার পরে, আপনার অ্যাকাউন্টে লগ ইন করুন এবং API অংশ অথবা ড্যাশবোর্ডে যান। নতুন API কী তৈরি বা জেনারেট করার বিকল্প খুঁজুন।
API কী আপনার .env ফাইলে কপি করুন।


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: এটি একটি পরিবর্তনশীল যা SERP API এন্ডপয়েন্টের জন্য বেস URL সংরক্ষণ করে। পরিবর্তনশীল নাম BASE_URL একটি রীতিনীতি যা নির্দেশ করে যে এই URL টি API অনুরোধগুলি করার শুরু পয়েন্ট।

'https://serpapi.com/search':

এটি প্রকৃত URL স্ট্রিং যা BASE_URL পরিবর্তনশীলের সাথে নির্ধারিত। এটি SERP API ব্যবহার করে সার্চ প্রশ্নসমূহ পরিচালনার জন্য এন্ডপয়েন্ট উপস্থাপন করে। 

# Purpose:
এই লাইনের উদ্দেশ্য হল একটি ধ্রুবক সংজ্ঞায়িত করা যা SERP API এর জন্য বেস URL ধারণ করে। এই URL API অনুরোধ তৈরি করার জন্য শুরু পয়েন্ট হিসেবে ব্যবহৃত হবে সার্চ অপারেশন কার্যকর করার জন্য।

# Usage:
বেস URL একটি পরিবর্তনশীল হিসেবে সংজ্ঞায়িত করার মাধ্যমে, আপনি আপনার কোড জুড়ে এটি সহজেই পুনরায় ব্যবহার করতে পারবেন যখনই আপনাকে SERP API তে অনুরোধ করতে হবে। এটি আপনার কোডকে আরও রক্ষণাবেক্ষণযোগ্য করে তোলে এবং একাধিক স্থানে URL হার্ডকোড করার কারণে ত্রুটির ঝুঁকি কমায়। বর্তমান উদাহরণটি হলো https://serpapi.com/search?engine=bing যা Bing সার্চ API ব্যবহার করছে। বিভিন্ন API নির্বাচন করা যেতে পারে https://Serpapi.com এ।


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# ব্যাখ্যা:

এখানেই আপনার প্লাগইন কোড অবস্থিত।

ক্লাস সংজ্ঞা: `class BookingPlugin`: BookingPlugin নামে একটি ক্লাস সংজ্ঞায়িত করে যা হোটেল এবং ফ্লাইট বুকিংয়ের জন্য মেথডগুলো ধারণ করে।

হোটেল বুকিং মেথড:

- `@kernel_function(description="booking hotel")`: একটি ডেকোরেটর যা ফাংশনটিকে হোটেল বুকিংয়ের জন্য একটি কার্নেল ফাংশন হিসেবে বর্ণনা করে।
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: হোটেল বুকিংয়ের জন্য একটি মেথড সংজ্ঞায়িত করে যার প্যারামিটার এবং রিটার্ন টাইপ এনোটেড।

মেথডটি হোটেল বুকিং অনুরোধের জন্য প্যারামিটারগুলোর একটি ডিকশনারি তৈরি করে এবং SERP API-তে একটি GET অনুরোধ পাঠায়। এটি রেসপন্স স্টেটাস পরীক্ষা করে এবং সফল হলে হোটেলের তথ্য প্রদান করে, অন্যথায় অনুরোধ ব্যর্থ হলে None ফেরত দেয়।

ফ্লাইট বুকিং মেথড:

- `@kernel_function(description="booking flight")`: একটি ডেকোরেটর যা ফাংশনটিকে ফ্লাইট বুকিংয়ের জন্য একটি কার্নেল ফাংশন হিসেবে বর্ণনা করে।
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: ফ্লাইট বুকিংয়ের জন্য একটি মেথড সংজ্ঞায়িত করে যার প্যারামিটার এবং রিটার্ন টাইপ এনোটেড।

মেথডটি আউটবাউন্ড এবং রিটার্ন ফ্লাইট অনুরোধের জন্য প্যারামিটারগুলোর ডিকশনারি তৈরি করে এবং SERP API-তে GET অনুরোধ পাঠায়। এটি রেসপন্স স্টেটাস পরীক্ষা করে এবং সফল হলে ফলাফল স্ট্রিংয়ে ফ্লাইট তথ্য সংযুক্ত করে, অন্যথায় ত্রুটির বার্তা প্রিন্ট করে। মেথডটি ফ্লাইট তথ্যসহ ফলাফল স্ট্রিংটি রিটার্ন করে।


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# ব্যাখ্যা:
ইমপোর্ট স্টেটমেন্টস: Azure শর্টকাটের জন্য প্রয়োজনীয় মডিউলসমূহ, AI এজেন্ট, চ্যাট মেসেজ কনটেন্ট, লেখকের ভূমিকা, এবং কার্নেল ফাংশন ডেকোরেটর ইমপোর্ট করুন।

অ্যাসিঙ্ক্রোনাস কনটেক্সট ম্যানেজার: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): এটি একটি অ্যাসিঙ্ক্রোনাস কনটেক্সট ম্যানেজার সেটআপ করে যা Azure শর্টকাট পরিচালনা করে এবং একটি AI এজেন্ট ক্লায়েন্ট তৈরি করে।

এজেন্টের নাম এবং নির্দেশাবলী:
- `AGENT_NAME = "BookingAgent"`: এজেন্টের নাম নির্ধারণ করে।
- `AGENT_INSTRUCTIONS = """..."""`: বুকিং অনুরোধ পরিচালনা করার জন্য এজেন্টের জন্য বিস্তারিত নির্দেশাবলী প্রদান করে।

এজেন্ট ডেফিনিশন তৈরি: `agent_definition = await client.agents.create_agent(...)`: নির্দিষ্ট মডেল, নাম, এবং নির্দেশাবলী দিয়ে একটি এজেন্ট ডেফিনিশন তৈরি করে।

AzureAI এজেন্ট তৈরি: `agent = AzureAIAgent(...)`: ক্লায়েন্ট, এজেন্ট ডেফিনিশন, এবং সংজ্ঞায়িত প্লাগইন ব্যবহার করে একটি AzureAI এজেন্ট তৈরি করে।

থ্রেড তৈরি: `thread: AzureAIAgentThread | None = None`: এজেন্টের জন্য একটি থ্রেড তৈরি করুন। প্রথমে থ্রেড তৈরি করা আবশ্যক নয় - যদি `None` মান প্রদান করা হয়, তবে প্রথম ইনভোকেশনের সময় একটি নতুন থ্রেড তৈরি হবে এবং প্রতিক্রিয়ার অংশ হিসেবে ফেরত দেওয়া হবে।

ব্যবহারকারীর ইনপুট: `user_inputs = ["..."]`: এজেন্টকে প্রক্রিয়াকরণের জন্য ব্যবহারকারীর ইনপুট তালিকা নির্ধারণ করে।

`finally` ব্লকে, থ্রেড এবং এজেন্ট মুছে ফেলে রিসোর্সগুলি পরিষ্কার করুন।


# Authentication

`DefaultAzureCredential` ক্লাসটি Python-এর Azure SDK-এর অংশ। এটি Azure সার্ভিসগুলোর সাথে প্রমাণীকরণ করার জন্য একটি ডিফল্ট পদ্ধতি প্রদান করে। এটি একটি নির্দিষ্ট ধারাবাহিকতায় একাধিক পদ্ধতি ব্যবহার করে প্রমাণীকরণ করার চেষ্টা করে, যেমন পরিবেশ ভেরিয়েবল, ম্যানেজড আইডেন্টিটি, এবং Azure CLI ক্রেডেনশিয়াল।

Asynchronous Operations: aio মডিউল নির্দেশ করে যে DefaultAzureCredential ক্লাসটি অ্যাসিনক্রোনাস অপারেশন সমর্থন করে। এর মানে আপনি এটি asyncio-এর সাথে ব্যবহার করতে পারেন যাতে নন-ব্লকিং প্রমাণীকরণ অনুরোধ করতে পারেন।


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকারকরণ**:  
এই নথিটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনূদিত হয়েছে। আমরা যথাসম্ভব সঠিকতার চেষ্টা করি, তবে স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে তা দয়া করে জানুন। মূল নথির নিজের ভাষায় থাকা তথ্যই কর্তৃত্বপ্রাপ্ত উৎস হিসেবে গণ্য করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদের ব্যবহার থেকে সৃষ্ট কোনো ভুলবোঝাবুঝি বা ভুল অর্থবোধের জন্য আমরা দায়ী নই।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
